Building the model

In [13]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import keras.utils as image
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator

In [14]:
import tensorflow as tf
from keras import layers, Model
from keras.applications import VGG16
from keras.optimizers import Adam

In [15]:
CSV_PATH = "ground_truth.csv"
IMAGE_FOLDER = "data/"

Ucitavanje podataka

In [16]:
# Učitavanje CSV-a
df = pd.read_csv(CSV_PATH)

In [17]:
def create_model(lr=0.0001):
    # Učitavanje pretreniranog modela VGG16 bez gornjeg sloja (include_top=False)
    base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    
    # Zamrzavanje slojeva pretreniranog modela
    for layer in base_model.layers:
        layer.trainable = False

    # Ako je fine_tune_at definisan, otključavamo gornji deo mreže
    # if fine_tune_at:
    #     for layer in base_model.layers[fine_tune_at:]:
    #         layer.trainable = True
    
    # Dodavanje novih slojeva prilagođenih binarnoj klasifikaciji
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)  # Dodajemo dropout za regularizaciju
    output = layers.Dense(1, activation='sigmoid')(x)
    
    # Kreiranje modela
    model = Model(inputs=base_model.input, outputs=output)
    
    # Kompajliranje modela
    model.compile(
        loss=tf.keras.losses.BinaryFocalCrossentropy(alpha=0.25, gamma=2.0),
        optimizer=Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    
    print(model.summary())
    return model


In [18]:
# Kreiranje modela
model = create_model()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   